In [1]:
import ee
import geemap
import json
import os
#from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import segment_testing_my_gee_functions as mgf

In [2]:
ee.Authenticate()
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

In [3]:
shp_folder_path = 'E:/!!Research/!!!Data/example_data/example_data_py/example_data_shp/'
shp_file = 'example_data.shp'
example_data = geemap.shp_to_ee(f'{shp_folder_path}{shp_file}')

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# checking that the imported layer loadedn
Map.addLayer(example_data)

In [6]:
naip = mgf.naip_func(example_data)

vis_param_naip = {'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
Map.addLayer(naip, vis_param_naip, f'naip')

savi = mgf.naip_savi_endvi(naip)
vis_param_savi = {'bands' : ['savi'], 
               'min' : 0, 
               'max' : 1,
               'gamma' : 1}

"""
endvi = mgf.naip_endvi(savi)
vis_param_savi = {'bands' : ['endvi'], 
               'min' : 0, 
               'max' : 1,
               'gamma' : 1}
"""
Map.addLayer(savi, vis_param_savi, f'savi')

In [26]:
#dog = mgf.dog_sharp(naip)

dog = mgf.dog_sharp(savi)

def gauss_three_func(given_image):
    ### gaussian smoothing
    gaussianKernel = ee.Kernel.gaussian(
      radius = 3, # was originally 3, also tested 10
      units = 'pixels'
    )
    
    gaussian_smooth = given_image.convolve(gaussianKernel)
    return gaussian_smooth

gauss_three = gauss_three_func(dog)
vis_param_smooth = {'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}
Map.addLayer(gauss_three, vis_param_smooth, f'smoothed three')

conversion of color NAIP to radiance grayscale image

In [18]:
grayscale = naip.expression(
      '(0.3 * R) + (0.59 * G) + (0.11 * B)', {
      'R': naip.select(['R']),
      'G': naip.select(['G']),
      'B': naip.select(['B'])
})

vis_param_gray = {#'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
#Map.addLayer(grayscale, vis_param_gray, f'gray')

# if wanting to use a grayscale image
int_gray = grayscale.int()
glcmTexture = int_gray.glcmTexture(5)

# if not wanting to use a grayscaled image
#int_naip = naip.int()
#glcmTexture = int_naip.glcmTexture(10)

imp_glcm_bands = glcmTexture.select(['constant_savg', 'constant_contrast'])
contrastVis = {
   'bands' : ['constant_savg', 'constant_contrast']
}

#Map.addLayer(imp_glcm_bands, contrastVis, 'contrast')

glcm_segment = imp_glcm_bands.addBands(gauss, ['R', 'G', 'B', 'N', 'savi']) #may need to include 'savi' and 'endvi'
contrastVis_rgb = {
   'bands' : ['R', 'G', 'B']
}

#Map.addLayer(glcm_segment, contrastVis_rgb, 'contrast_rgb')

standardized = mgf.stdrd_func(glcm_segment, example_data)
vis_param_std = {'bands' : ['N', 'R', 'constant_savg'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}
#Map.addLayer(standardized, vis_param_std, 'standardized')

In [21]:
def segmentation_func(stdrd_image, seed_spacing = 5, snic_compactness = 0, snic_connectivity = 4, seed_grid_shape = 'hex'):
    """
    Creates a segmented image using the script shown in the GEE workshop video.
    May add the option to change the SNIC function settings"""
    #standardized = stdrd_image
    #gauss = gauss_smooth_func(standardized)
    #gauss = dog_sharp(gauss)
    
    bands = ['constant_savg', 'constant_contrast', 
             'R', 'G', 'B', 'N'] #may need to include 'savi' and 'endvi'
    # seed_spacing default is now 5
    # seed_grid_shape now defaults to 'hex'
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(seed_spacing, seed_grid_shape)
    
    # Run SNIC on the regular square grid.
    snic = ee.Algorithms.Image.Segmentation.SNIC(
      image = stdrd_image,
      #size = 4, # shouldn't matter since have seed image
      compactness = snic_compactness, # spatial weighting not taken into account at 0. More compact at higher values
      connectivity = snic_connectivity, # 4 or 8
      neighborhoodSize = 2 * seed_spacing,
      seeds = seeds # the "seedGrid" given
    ).select(['constant_savg_mean', 'constant_contrast_mean',
              'R_mean', 'G_mean', 'B_mean', 'N_mean', 'savi_mean', 'clusters'], #may need to include 'savi' and 'endvi'
             ['constant_savg', 'constant_contrast',
              'R', 'G', 'B', 'N', 'savi', 'clusters'])
    
    clusters = snic.select('clusters')
    stdDev = stdrd_image.addBands(clusters).reduceConnectedComponents(ee.Reducer.stdDev(), 'clusters', 256)
    area = ee.Image.pixelArea().addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256)
    minMax = clusters.reduceNeighborhood(ee.Reducer.minMax(), ee.Kernel.square(1))
    perimeterPixels = minMax.select(0).neq(minMax.select(1)).rename('perimeter')
    perimeter = perimeterPixels.addBands(clusters) \
        .reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256)
    sizes = ee.Image.pixelLonLat().addBands(clusters).reduceConnectedComponents(ee.Reducer.minMax(), 'clusters', 256)
    width = sizes.select('longitude_max').subtract(sizes.select('longitude_min')).rename('width')
    height = sizes.select('latitude_max').subtract(sizes.select('latitude_min')).rename('height')
    objectPropertiesImage = ee.Image.cat([
      snic.select(bands),
      stdDev,
      area,
      perimeter,
      width,
      height
    ]).float()
    return objectPropertiesImage

Using the above function

In [22]:
segmented_image = segmentation_func(standardized, seed_spacing = 16, snic_compactness = 0, 
                                                     snic_connectivity = 4, seed_grid_shape = 'square')
Map.addLayer(segmented_image.randomVisualizer(), {}, 'clusters')

In [23]:
max_clusters = 10 # tested with 10
obj_classed = mgf.obj_class(example_data, segmented_image, 
                                         train_scale = 2, train_pixels = 10000, #default was 10000
                                         max_clusters = max_clusters, tilescale = 5, segmented_as_bands = False, 
                                         band_list = None)
obj_palette = ['#000000','#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']
obj_vis = {'min' : 0, 'max' : 12, 'palette' : obj_palette}
Map.addLayer(obj_classed, obj_vis, f'object based classification with poly_1 and {max_clusters} clusters')

In [21]:
shp = example_data
shp_geom = shp.geometry()
exp_file_dir = 'E:/!!Research/!!!Data/example_data/example_data_py/classified/ex_obj_classed_10_no_savi.tif'

geemap.ee_export_image(
    obj_classed, filename = exp_file_dir, scale=1, region=shp_geom, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to E:\!!Research\!!!Data\example_data\example_data_py\classified\ex_obj_classed_10_no_savi.tif


In [12]:
shp = example_data
shp_geom = shp.geometry()
exp_file_dir_naip = 'E:/!!Research/!!!Data/example_data/example_data_py/naip/ex_naip.tif'

geemap.ee_export_image(
    naip, filename = exp_file_dir_naip, scale=1, region=shp_geom, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to E:\!!Research\!!!Data\example_data\example_data_py\classified\ex_naip.tif
